# OMGeEP

# Download dependencies

In [ ]:
# %pip install lightgbm scikit-learn pandas numpy joblib
# %pip install iterative-stratification
# %pip install shap

# Import libraries

In [19]:
import ast
import re
import os
import numpy as np
import pandas as pd
import math

from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score, hamming_loss, roc_auc_score, precision_recall_fscore_support
from lightgbm import LGBMClassifier
from joblib import Parallel, delayed

# Optional iterative stratification (better for multilabel splits)
try:
    from iterstrat.ml_stratifiers import iterative_train_test_split
    ITERATIVE_AVAILABLE = True
except Exception:
    ITERATIVE_AVAILABLE = False
    print("iterative-stratification not available — splits will be random.")


import warnings
warnings.filterwarnings('ignore')

iterative-stratification not available — splits will be random.


# Global variables

In [32]:
OUTPUT_DIR = 'ifbdata/atlanteco_hack/OMGeEP/output_files'
SAMPLE_ID_COL = 'sample_id'
SITE_ID_COL = 'sampling_design_feature'

# General_Helper_functions

In [95]:
def get_most_variable_omics(df:pd.DataFrame, N:int=200, ID_col:str=SAMPLE_ID_COL) -> pd.DataFrame:
    """
    df: metagenomic gene df
    N: Top N most variable genes
    """
    copied_df = df.copy()
    copied_df.set_index(ID_col,inplace=True)
    gene_variance = copied_df.var(axis=1)
    # Rank genes by variance
    top_X = N   # choose the number of genes you want
    top_genes = gene_variance.nlargest(top_X).index
    copied_df_top = copied_df.loc[top_genes]
    X = copied_df_top.T
    return X

In [69]:
def create_sample_id_mapping(file_path:str) -> pd.DataFrame:
    mapping_file = pd.read_csv(file_path)
    mapping_file = mapping_file[['sample-id', 'station-label']]
    mapping_file.rename(columns={'sample-id':SAMPLE_ID_COL, 'station-label':SITE_ID_COL}, inplace=True)
    return mapping_file

In [70]:
mapping_file = create_sample_id_mapping('Sample_ID_mapping.csv')
mapping_file.head()

,sample_id,sampling_design_feature
0,SAMEA7950653,MMA-071
1,SAMEA7950654,MMA-071
2,SAMEA7950655,MMA-071
3,SAMEA7950656,MMA-071
4,SAMEA7918377,MMA-071


# Read genomic data

In [79]:
ben_mag_df = pd.read_csv('ifbdata/atlanteco_hack/MetaGenomics/BenguelaCurrent_MAGab/BenguelaCurrent_SylphResults.tsv',sep="\t")

In [81]:
ben_mag_df.head()

,Unnamed: 0,key,ProfilingMethod,Sample-ID,Station-Label,TopicalStudy,Sample_file,Genome_file,Taxonomic_abundance,Sequence_abundance,...,Eff_lambda,Lambda_5-95_percentile,Median_cov,Mean_cov_geq1,Containment_ind,Naive_ANI,kmers_reassigned,Contig_name,GTDB_classification,NCBI_classification
0,0,SAMEA7934747,sylph.v0.8.0,SAMEA7934747,MMA-123,MMA-Topical-Study_Benguela-Current-Ecosystem_L...,ERR14792484_1.fastq.gz,ERR14792484_concoct_104,30.8299,2.1145,...,HIGH,NA-NA,196.0,196.961,14025/14027,100.00,1.0,NODE.4_length_332230_cov_108.525819,168 d__Bacteria;p__Bacteroidota;c__Bacteroi...,168 d__Bacteria;p__Bacteroidetes;c__Flavoba...
1,1,SAMEA7934747,sylph.v0.8.0,SAMEA7934747,MMA-123,MMA-Topical-Study_Benguela-Current-Ecosystem_L...,ERR14792484_1.fastq.gz,ERR14792476_binner12_Refined_88,8.4939,0.6304,...,HIGH,NA-NA,54.0,60.522,15302/15390,99.98,26.0,NODE.6_length_154883_cov_8.411431,108 d__Bacteria;p__Pseudomonadota;c__Gammap...,108 d__Bacteria;p__Proteobacteria;c__Gammap...
2,2,SAMEA7934747,sylph.v0.8.0,SAMEA7934747,MMA-123,MMA-Topical-Study_Benguela-Current-Ecosystem_L...,ERR14792484_1.fastq.gz,ERR14792494_binner12_Refined_174,7.8648,0.5930,...,HIGH,NA-NA,50.0,47.963,15378/15594,99.96,89.0,NODE.77_length_137683_cov_7.341159,118 d__Bacteria;p__Pseudomonadota;c__Gammap...,118 d__Bacteria;p__Proteobacteria;c__Gammap...
3,3,SAMEA7934747,sylph.v0.8.0,SAMEA7934747,MMA-123,MMA-Topical-Study_Benguela-Current-Ecosystem_L...,ERR14792484_1.fastq.gz,ERR14792499_binner123_Refined_14,6.2918,0.4557,...,HIGH,NA-NA,40.0,40.616,15081/15095,100.00,2.0,NODE.5_length_393469_cov_9.236133,124 d__Bacteria;p__Pseudomonadota;c__Gammap...,124 d__Bacteria;p__Proteobacteria;c__Gammap...
4,4,SAMEA7934747,sylph.v0.8.0,SAMEA7934747,MMA-123,MMA-Topical-Study_Benguela-Current-Ecosystem_L...,ERR14792484_1.fastq.gz,ERR14792491_binner13_Refined_82,5.3480,0.3840,...,HIGH,NA-NA,34.0,33.151,14499/14803,99.93,21.0,NODE.196_length_68065_cov_7.101867,220 d__Bacteria;p__Pseudomonadota;c__Alphap...,220 d__Bacteria;p__Proteobacteria;c__Alphap...


## Helper functions

In [3]:
def normalize_gene_abundances(df:pd.DataFrame, method:str='tss', id_col:str=None) -> pd.DataFrame:
    """
    Normalize gene abundance data for comparison between samples.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame with gene IDs in first column and samples as remaining columns
    method : str, default 'tss'
        Normalization method:
        - 'tss': Total Sum Scaling (relative abundance, sums to 1)
        - 'tss_percent': Total Sum Scaling as percentages (sums to 100)
        - 'z_score': Z-score normalization (mean=0, std=1)
        - 'min_max': Min-max scaling (0 to 1 range)
        - 'log_tss': Log-transformed TSS (log10(TSS + pseudocount))
        - 'clr': Centered Log Ratio transformation
    id_col : str, optional
        Name of ID column. If None, assumes first column is ID
        
    Returns:
    --------
    pandas.DataFrame
        Normalized DataFrame with same structure as input
    """
    
    # Make a copy to avoid modifying original
    df_norm = df.copy()
    
    # Identify ID column
    if id_col is None:
        id_col = df.columns[0]
    
    # Get sample columns (all except ID column)
    sample_cols = [col for col in df.columns if col != id_col]
    
    # Extract abundance matrix
    abundance_matrix = df_norm[sample_cols].values
    
    if method == 'tss':
        # Total Sum Scaling - convert to relative abundances
        col_sums = abundance_matrix.sum(axis=0)
        normalized_matrix = abundance_matrix / col_sums
        
    elif method == 'tss_percent':
        # Total Sum Scaling as percentages
        col_sums = abundance_matrix.sum(axis=0)
        normalized_matrix = (abundance_matrix / col_sums) * 100
        
    elif method == 'z_score':
        # Z-score normalization (standardization)
        normalized_matrix = (abundance_matrix - abundance_matrix.mean(axis=0)) / abundance_matrix.std(axis=0)
        
    elif method == 'min_max':
        # Min-max scaling to [0, 1] range
        min_vals = abundance_matrix.min(axis=0)
        max_vals = abundance_matrix.max(axis=0)
        normalized_matrix = (abundance_matrix - min_vals) / (max_vals - min_vals)
        
    elif method == 'log_tss':
        # Log-transformed TSS (common in metagenomics)
        col_sums = abundance_matrix.sum(axis=0)
        tss_matrix = abundance_matrix / col_sums
        # Add small pseudocount to avoid log(0)
        pseudocount = 1e-10
        normalized_matrix = np.log10(tss_matrix + pseudocount)
        
    elif method == 'clr':
        # Centered Log Ratio transformation
        # Add small pseudocount to avoid log(0)
        pseudocount = 1e-10
        log_matrix = np.log(abundance_matrix + pseudocount)
        geometric_means = log_matrix.mean(axis=0)
        normalized_matrix = log_matrix - geometric_means
        
    else:
        raise ValueError(f"Unknown normalization method: {method}")
    
    # Replace the sample columns with normalized values
    df_norm[sample_cols] = normalized_matrix
    
    return df_norm
    

In [40]:
def read_genomic_data(genomic_data_path:str, rows_to_skip:int=7) -> pd.DataFrame:

    # read df
    gen_df = pd.read_csv(genomic_data_path, sep='\t')
    # rename gene id row to ID
    gen_df.rename(columns={'Unnamed: 0':SAMPLE_ID_COL}, inplace=True)
    # remove metaparams
    gen_df = gen_df.iloc[rows_to_skip:]
    # convert NAN to 0
    gen_df.fillna(0, inplace=True)
    # change values to numeric (expect geneID)
    sample_cols = gen_df.columns.drop(SAMPLE_ID_COL)
    gen_df[sample_cols] = gen_df[sample_cols].apply(pd.to_numeric, errors='coerce')

    # Remove samples with no genes
    # 1. Calculate column sums for sample columns
    col_sums = gen_df[sample_cols].sum()
    # 2. Find columns with zero sum
    zero_sum_cols = col_sums[col_sums == 0].index.tolist()
    # 3. Remove zero-sum columns
    if zero_sum_cols:
        gen_df = gen_df.drop(columns=zero_sum_cols)

    # reset index
    gen_df.reset_index(drop=True, inplace=True)
    
    # normalize the results per sample
    gen_df_normalized = normalize_gene_abundances(gen_df, method='tss', id_col=SAMPLE_ID_COL)

    # return normalized df
    return gen_df_normalized

## Running genomic code

In [92]:
ben_gen_df = read_genomic_data(genomic_data_path='ifbdata/atlanteco_hack/MetaGenomics/BenguelaCurrent_GeneAb/BenguelaCurrent_ffn_GeneAb_T.tsv')

In [102]:
top_200_ben_genes = get_most_variable_omics(ben_gen_df, 200)
top_200_ben_genes.head()

sample_id,ERR13506499_binner13_Refined_13_00241,RD49_S11_L002_concoct_88_00609,ERR14792500_concoct_73_00952,ERR14792466_binner12_Refined_201_02069,ERR14792484_binner12_Refined_21_01647,ERR14792474_binner12_Refined_95_02500,ERR14792503_binner123_Refined_46_00236,ERR13506423_concoct_67_00051,ERR14792468_metabat2_10_01118,ERR14792474_binner12_Refined_58_01333,...,ERR14792508_concoct_23_00832,ERR14792494_concoct_68_00603,ERR14792499_binner12_Refined_62_02272,ERR14792508_concoct_23_00544,ERR14792494_concoct_68_00409,ERR14792494_concoct_68_00496,ERR14792494_concoct_68_00014,ERR14792494_concoct_68_00186,ERR14792494_concoct_68_00822,ERR14792472_binner13_Refined_56_00115
SAMEA7908697,0.015145,0.004184,0.000002,0.000000,0.000000e+00,0.000000,3.889218e-06,0.000758,0.000016,2.040104e-06,...,9.419175e-08,0.000000,1.111457e-05,0.000000,0.000000,4.330850e-07,0.000000,1.131218e-06,2.427111e-07,0.000220
SAMEA7909194,0.033343,0.009253,0.000000,0.000000,0.000000e+00,0.000000,4.874588e-07,0.007311,0.000003,5.710084e-07,...,1.323540e-05,0.000002,7.543917e-06,0.000003,0.000014,4.818135e-06,0.000000,1.643667e-05,7.650181e-07,0.000033
SAMEA7913651,0.794466,0.198864,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000
SAMEA7926718,0.072409,0.020711,0.000000,0.000000,3.016018e-07,0.000000,5.119132e-07,0.018887,0.000002,6.549923e-07,...,4.712454e-05,0.000009,6.050491e-06,0.000015,0.000118,7.360560e-05,0.000094,6.763142e-05,1.782212e-05,0.000003
SAMEA7896668,0.013803,0.003844,0.000000,0.000003,0.000000e+00,0.000004,1.346209e-06,0.001491,0.000005,8.691979e-06,...,0.000000e+00,0.000000,5.579185e-07,0.000000,0.000000,0.000000e+00,0.000000,2.408541e-08,0.000000e+00,0.000269


In [42]:
# wedd_gen_df = read_genomic_data(genomic_data_path='ifbdata/atlanteco_hack/MetaGenomics/WeddellSea_GeneAb/WeddellSea_ffn_GeneAb_T.tsv')

# Read environmental data

## Helper functions

In [23]:
def read_env_data(env_data_csv_path:str) -> pd.DataFrame:
    return pd.read_csv(env_data_csv_path)

In [24]:
env_df = read_env_data('machine_ready_env.csv')
env_df.head()

,Feature,MMA-071,MMA-072,MMA-074,MMA-076,MMA-077,MMA-078,MMA-079,MMA-080,MMA-081,...,MMA-118,MMA-120,MMA-121,MMA-122,MMA-123,MMA-124,MMA-127,MMA-130,MMA-131,MMA-132
0,CTD temperature (degC) median,-0.286,-0.1140,-1.0150,-0.8130,-0.9110,-0.9130,-0.943,-1.0330,-0.922,...,15.2730,17.1460,15.1610,16.6450,16.8800,13.9100,20.9720,24.0940,23.7460,24.2430
1,CTD salinity (psu) median,34.422,33.9444,33.0256,33.0732,33.1779,33.1789,33.167,33.2374,33.360,...,35.1678,35.5247,35.4309,35.3534,35.4706,35.3161,36.2856,29.0101,30.8759,36.0856
2,CTD oxygen (%) median,93.000,104.8000,95.2000,95.3000,94.6000,94.5000,94.600,93.9000,94.300,...,99.3000,89.3000,63.2000,97.1000,92.7000,34.7000,104.4000,79.8000,76.1000,80.7000
3,CTD fluorescence (mg/m3) median,123.569,48.7230,4.3520,1.8870,3.7230,2.8450,2.085,1.8150,2.734,...,9.0850,4.0080,4.3500,8.7580,5.6290,5.8940,5.1880,1.5250,1.8440,2.0290
4,NO2,0.150,0.0800,0.1900,0.2200,0.1900,0.2100,0.200,0.1700,0.170,...,0.4600,0.5000,0.5800,0.1700,0.3700,0.6200,0.1600,0.0000,0.0000,0.0100


# Read Proteomics data

## Helper function

In [99]:
def read_prot_data(prot_data_path:str) -> pd.DataFrame:
    df = pd.read_csv(prot_data_path)
    df.columns = df.columns.str.replace('_normalized', '')
    df.rename(columns={"MAG_name_prot":SAMPLE_ID_COL}, inplace=True)
    return df

## Running proteomic code

In [100]:
ben_prot_df = read_prot_data('annotation_normalized_benguela.csv')

In [101]:
top_200_ben_prots = get_most_variable_omics(ben_prot_df, 200)
top_200_ben_prots.head()

sample_id,megahit_HN020_k141_129271.p1,transabyss_HN031_.k50S2815693.p1,transabyss_HN017_.k50R6170578.p2,transabyss_HN006_.k50R5032280.p1,trinity_HN021_TRINITY_DN12475_c0_g1_i1.p3,megahit_HN002_k141_333561.p1,transabyss_HN026_.k50R3537400.p2,trinity_HN039_TRINITY_DN8611_c0_g1_i9.p1,megahit_HN002_k141_173328.p1,megahit_HN011_k141_216036.p1,...,trinity_HN002_TRINITY_DN12831_c0_g2_i2.p1,megahit_HN022_k141_181984.p1,megahit_HN016_k141_237681.p1,megahit_HN016_k141_148937.p1,transabyss_HN042_.k20R32033753.p1,trinity_HN011_TRINITY_DN58831_c0_g1_i1.p1,megahit_HN006_k141_164784.p1,megahit_HN011_k141_520560.p2,trinity_HN017_TRINITY_DN29133_c0_g1_i3.p2,trinity_HN038_TRINITY_DN19851_c0_g1_i1.p1
SAMEA7917467,1.380000e-07,0.000038,5.550000e-07,0.000002,0.000004,0.000005,9.430000e-07,2.610000e-06,0.000019,1.570000e-07,...,1.300000e-07,1.980000e-06,0.000001,6.190000e-07,8.150000e-07,0.000000e+00,3.680000e-07,0.0,3.660000e-06,0.000002
SAMEA7917491,8.690000e-07,0.000032,3.490000e-07,0.000000,0.000002,0.000007,3.920000e-06,4.680000e-07,0.000043,0.000000e+00,...,3.670000e-07,4.540000e-06,0.000003,5.890000e-07,1.260000e-06,6.950000e-08,2.740000e-07,0.0,6.450000e-06,0.000004
SAMEA7917515,1.360000e-07,0.000037,4.960000e-06,0.000004,0.000004,0.000003,1.070000e-06,1.290000e-06,0.000029,9.560000e-08,...,0.000000e+00,1.150000e-06,0.000002,3.970000e-07,1.420000e-06,1.740000e-07,7.100000e-07,0.0,5.010000e-06,0.000003
SAMEA7908692,0.000000e+00,0.000034,5.360000e-06,0.000002,0.000018,0.000008,5.780000e-07,2.330000e-06,0.000030,1.060000e-07,...,8.990000e-08,2.370000e-06,0.000004,7.080000e-07,1.550000e-06,2.340000e-08,3.070000e-07,0.0,4.300000e-07,0.000003
SAMEA7908700,6.560000e-08,0.000039,1.540000e-06,0.000000,0.000002,0.000000,1.260000e-06,4.560000e-06,0.000015,2.990000e-08,...,0.000000e+00,1.080000e-07,0.000001,8.830000e-08,4.670000e-07,2.850000e-08,1.300000e-07,0.0,1.790000e-07,0.000002


In [67]:
ben_prot_df.head()

,MAG_name_prot,SAMEA7917467,SAMEA7917491,SAMEA7917515,SAMEA7908692,SAMEA7908700,SAMEA7909095,SAMEA7909203,SAMEA7909267,SAMEA7913658,...,SAMEA7934767,SAMEA7934820,SAMEA7934998,SAMEA7914376,SAMEA7935115,SAMEA7939692,SAMEA7926962,SAMEA7939770,SAMEA7927003,SAMEA7927059
0,megahit_HN020_k141_129271.p1,1.380000e-07,8.690000e-07,1.360000e-07,0.000000e+00,6.560000e-08,4.430000e-08,7.800000e-08,0.000000e+00,3.280000e-08,...,0.000000e+00,1.470000e-07,1.830000e-08,0.000002,3.880000e-08,0.000000e+00,1.230000e-07,8.940000e-08,0.000332,3.690000e-07
1,trinity_HN006_TRINITY_DN216338_c0_g1_i1.p1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
2,megahit_HN025_k141_9493.p1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
3,transabyss_HN006_.k50R5032280.p1,2.390000e-06,0.000000e+00,3.810000e-06,2.200000e-06,0.000000e+00,2.880000e-07,0.000000e+00,0.000000e+00,0.000000e+00,...,1.940000e-06,3.380000e-06,1.270000e-06,0.000000,0.000000e+00,5.750000e-07,0.000000e+00,4.960000e-06,0.000150,5.680000e-06
4,transabyss_HN026_.k50R3537400.p2,9.430000e-07,3.920000e-06,1.070000e-06,5.780000e-07,1.260000e-06,6.910000e-08,3.480000e-07,7.590000e-07,2.940000e-07,...,3.950000e-07,8.660000e-07,1.780000e-07,0.000001,5.070000e-07,2.540000e-07,1.200000e-06,2.190000e-06,0.000119,1.580000e-06


# Read Metabolomic data (labels)

## Helper functions

In [5]:
def filter_relevant_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Keep only columns starting with 'featureId' or 'SAMEA'.
    """
    return df.loc[:, df.columns.str.startswith(('featureId', 'SAMEA'))]

In [6]:
def clean_samea_column_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove the trailing '_RX' from SAMEA column names.
    E.g., SAMEA123456_R01_R2 -> SAMEA123456_R01
    """
    rename_map = {
        col: re.sub(r'(SAMEA\d+_R\d+)_R\d+$', r'\1', col)
        if col.startswith('SAMEA') else col
        for col in df.columns
    }
    return df.rename(columns=rename_map)

In [7]:
def group_samea_columns(df: pd.DataFrame, threshold: float = 2e4) -> pd.DataFrame:
    """
    Group columns with the same SAMEAXXXXXX prefix:
      - If any column in the group > threshold → grouped value = 1
      - If all columns in the group ≤ threshold → grouped value = 0

    The grouped columns will replace the original SAMEA columns.
    """
    # Map each SAMEA column to its base prefix (SAMEAXXXXXX)
    prefix_map = {
        col: re.match(r'(SAMEA\d+)', col).group(1)
        if col.startswith('SAMEA') else col
        for col in df.columns
    }

    result = df.copy()
    for prefix in set(prefix_map.values()):
        if prefix.startswith('SAMEA'):
            same_cols = [col for col, pfx in prefix_map.items() if pfx == prefix]

            # 🔹 Ensure these columns are numeric (convert strings → numbers, non-numeric → NaN)
            numeric_block = df[same_cols].apply(pd.to_numeric, errors='coerce')

            # ✅ Compute on numeric_block, not df
            result[prefix] = (numeric_block > threshold).any(axis=1).astype(int)

            # Drop original group columns
            result = result.drop(columns=same_cols)

    return result


In [8]:
def process_metabolomic_data(metabolome_data_path:str, threshold: float = 2e4) -> pd.DataFrame:
    """
    Full pipeline:
      1. Filter columns
      2. Clean SAMEA column names
      3. Group SAMEA columns using threshold logic
    """
    metabolome_df = pd.read_csv(metabolome_data_path, sep='\t')
    metabolome_df.drop(metabolome_df.tail(1).index,inplace=True) # drop last row
    metabolome_df = filter_relevant_columns(metabolome_df)
    metabolome_df = clean_samea_column_names(metabolome_df)
    metabolome_df = group_samea_columns(metabolome_df, threshold=threshold)
    return metabolome_df

## Running metabolomic code

In [9]:
metabolome_path = 'ifbdata/atlanteco_hack/MetaMetabolomics/1_Feature_table_univariate_analysis_hackathon'

processed_metabolomic_data = process_metabolomic_data(metabolome_path)

In [10]:
sample_cols = [col for col in processed_metabolomic_data.columns if col != 'featureId']
counts = processed_metabolomic_data[sample_cols].stack().value_counts()
print(counts)

1    651561
0     44715
Name: count, dtype: int64


In [11]:
len(processed_metabolomic_data['featureId'].unique())

1842

In [68]:
processed_metabolomic_data.head()

,featureId,SAMEA7951578,SAMEA7940389,SAMEA7920182,SAMEA7920184,SAMEA7928500,SAMEA7908666,SAMEA7955281,SAMEA7948843,SAMEA7904744,...,SAMEA7908718,SAMEA7935443,SAMEA7928718,SAMEA7950917,SAMEA7951482,SAMEA7904605,SAMEA7928635,SAMEA7909128,SAMEA7954424,SAMEA7927092
0,FT00002,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,FT00005,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
2,FT00017,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,FT00112,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,FT00326,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


# Mapping samples to sites

In [110]:
mapping = dict(zip(mapping_file['sample_id'], mapping_file['sampling_design_feature']))

In [113]:
top_200_ben_prots.index = top_200_ben_prots.index.map(mapping)
top_200_ben_genes.index = top_200_ben_genes.index.map(mapping)

In [122]:
env_df_t = env_df.T
env_df_t.columns = env_df_t.iloc[0]
env_df_t = env_df_t.drop(env_df_t.index[0])

In [129]:
# Aggregate by mean
# TODO - why do we have that? only ok if these are replicates
top_200_ben_genes = top_200_ben_genes.groupby(top_200_ben_genes.index).mean()
top_200_ben_prots = top_200_ben_prots.groupby(top_200_ben_prots.index).mean()


# Run statistical model

In [127]:
common_sites = top_200_ben_genes.index.intersection(top_200_ben_prots.index).intersection(env_df_t.index)

top_200_ben_genes_shared_sites = top_200_ben_genes.loc[common_sites]
top_200_ben_prots_shared_sites = top_200_ben_prots.loc[common_sites]
env_features_shared_sites = env_df_t.loc[common_sites]

full_feature_table = pd.concat([top_200_ben_genes_shared_sites, top_200_ben_prots_shared_sites, env_features_shared_sites], axis=1)

In [ ]:
metabolomic_data_t = processed_metabolomic_data.T
metabolomic_data_t.columns = metabolomic_data_t.iloc[0]
metabolomic_data_t = metabolomic_data_t.drop(metabolomic_data_t.index[0])

In [ ]:
metabolomic_data_t.index = metabolomic_data_t.index.map(mapping)
metabolomic_data_shared_sites = metabolomic_data_t.loc[common_sites]

In [ ]:
metabolomic_data_shared_sites = metabolomic_data_shared_sites.groupby(metabolomic_data_shared_sites.index).max()
metabolomic_data_shared_sites

In [137]:
counts = metabolomic_data_shared_sites.values.flatten()
pd.Series(counts).value_counts().sort_index()

0      545
1    17875
Name: count, dtype: int64

# Run code

In [139]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [140]:
X = full_feature_table
Y = metabolomic_data_shared_sites

In [147]:
# Get column names that contain NaN
cols_with_nan = X.columns[X.isnull().any()].tolist()
print(f"Columns with NaN: {cols_with_nan}")

Columns with NaN: ['Sum_Carotenes', 'Chlorophyll_a']


In [149]:
X['Chlorophyll_a']

MMA-106    3.6627
MMA-108    0.9782
MMA-114    0.5784
MMA-115    1.0038
MMA-123    0.4749
MMA-124    1.6468
MMA-127       NaN
MMA-130       NaN
MMA-131    0.2128
MMA-132    0.2364
Name: Chlorophyll_a, dtype: object

In [148]:
X['Sum_Carotenes']

MMA-106    0.0819
MMA-108     0.039
MMA-114    0.0145
MMA-115     0.033
MMA-123    0.0062
MMA-124    0.0546
MMA-127       NaN
MMA-130       NaN
MMA-131    0.0052
MMA-132    0.0156
Name: Sum_Carotenes, dtype: object

In [141]:
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), index=X.index, columns=X.columns)

In [142]:
# Dictionary to store top features per metabolite
top_features_per_metabolite = {}

In [143]:
# Loop over each metabolite
for metabolite in Y.columns:
    y = Y[metabolite].values
    
    # Skip metabolites that are all 0 or all 1
    if len(np.unique(y)) < 2:
        continue
    
    # ElasticNet logistic regression
    clf = LogisticRegression(
        penalty='elasticnet',
        solver='saga',
        l1_ratio=0.8,   # adjust between 0 (Ridge) and 1 (Lasso)
        C=1.0,          # inverse regularization strength
        max_iter=5000
    )
    
    # Fit model
    clf.fit(X_scaled, y)
    
    # Get absolute coefficients and sort
    coefs = pd.Series(np.abs(clf.coef_.ravel()), index=X_scaled.columns)
    top_features = coefs.sort_values(ascending=False).head(10).index.tolist()
    
    top_features_per_metabolite[metabolite] = top_features

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values